## Dependencies

In [137]:
import matplotlib.pyplot as plt
import networkx as nx # for graph creation
import time # for timing
from itertools import permutations # for generating all possible permutations


## Graph Creation

In [138]:
class Graph():
    def __init__(self, vertices):
        # Initialize the graph as an adjacency matrix with all with 0's
        self.graph = [[0 for column in range(vertices)] for row in range(vertices)]
        self.V = vertices  # Set the number of vertices

    def add_edge(self, u, v):
        # Add an undirected edge between u and v
        self.graph[u][v] = 1
        self.graph[v][u] = 1

    def add_edges_from_graph(self, graph):
        for edge in graph.edges():
            self.add_edge(edge[0], edge[1])

    def summary(self):
        # print nodes and edges
        print("Nodes: ", self.V)
        print("Edges: ", sum([sum(row) for row in self.graph])//2)

    def plot_graph(self):
        G = nx.Graph()
        for i in range(self.V):
            G.add_node(i)
        for i in range(self.V):
            for j in range(i, self.V):
                if self.graph[i][j] == 1:
                    G.add_edge(i, j)
        nx.draw(G, with_labels=True)
        plt.show()

In [ ]:
# make graph
n = 10
cycle_graph = nx.complete_graph(n)
cycle_graph_instance = Graph(n)
cycle_graph_instance.add_edges_from_graph(cycle_graph)
cycle_graph_instance.summary()

## Brute Force

In [139]:
def is_valid_cycle(perm, adj):
    # Check if the permutation forms a valid cycle in the graph
    N = len(perm)
    # Check if all consecutive vertices are connected
    for i in range(N - 1):
        if adj[perm[i]][perm[i + 1]] == 0:
            return False
    # Check if the last vertex is connected to the first vertex to complete the cycle
    if adj[perm[N-1]][perm[0]] == 0:
        return False
    return True

def hamiltonian_cycle_bruteforce(graph):
    # Find a Hamiltonian cycle in the graph using brute force
    adj = graph.graph if isinstance(graph, Graph) else graph  # Assuming Graph class has a 'graph' attribute
    N = len(adj)  # Number of vertices
    all_perms = permutations(range(N))
    
    for perm in all_perms:
        if is_valid_cycle(perm, adj):
            complete_cycle = perm + (perm[0],)
            print_bf(complete_cycle)
            plot_bf(graph, complete_cycle)
            return True  # Return True and the complete cycle
    plot_bf(graph)
    print("Solution does not exist\n")
    return False  # Return False if no valid cycle is found

def print_bf(path):
    print("Solution Exists: Following is one Hamiltonian Cycle")
    for vertex in path[:-1]:
        print(vertex, end=" -> ")
    print(path[-1])

def plot_bf(graph, path=None):
    G = nx.Graph()
    for i in range(len(graph)):
        for j in range(i, len(graph)):
            if graph[i][j] == 1:
                G.add_edge(i, j)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(5, 2))
    nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='yellow', edge_color='gray')
    if path:
        edges_in_path = [(path[i], path[(i + 1) % len(graph)]) for i in range(len(graph))]
        nx.draw_networkx_edges(G, pos, edgelist=edges_in_path, edge_color='red', width=2)
    plt.show()

In [ ]:
hamiltonian_cycle_bruteforce(cycle_graph_instance.graph)

## Back Tracking

In [140]:
def isSafe(graph, v, pos, path):
    # Check if current vertex and last vertex in path are adjacent
    if graph[path[pos-1]][v] == 0:
        return False
    # Check if current vertex not already in path
    for vertex in path:
        if vertex == v:
            return False
    return True

def hamCycleUtil(graph, path, pos):
    V = len(graph)
    # Base case: if all vertices are included in the path
    if pos == V:
        # Last vertex must be adjacent to the first vertex in path to make a cycle
        if graph[path[pos-1]][path[0]] == 1:
            return True
        else:
            return False
    # Explore all vertices other than the first vertex
    for v in range(1, V):
        if isSafe(graph, v, pos, path):
            path[pos] = v
            if hamCycleUtil(graph, path, pos+1):
                return True
            # DeadEnd - Remove current vertex if it doesn't lead to a solution
            path[pos] = -1
    return False

def hamCycle(graph):
    V = len(graph)
    # Initialize path as -1
    path = [-1] * V
    # Start from vertex 0 as the first vertex in the path
    path[0] = 0
    # Call the recursive helper function to find Hamiltonian cycle
    if not hamCycleUtil(graph, path, 1):
        print("Solution does not exist\n")
        plot_bt(graph)
        return False
    print_bt(path)
    plot_bt(graph, path)
    return True

def print_bt(path):
    print("Solution Exists: Following is one Hamiltonian Cycle")
    for vertex in path:
        print(vertex, end=' -> ')
    print(path[0])  # Print the first vertex again to show the complete cycle

def plot_bt(graph, path=None):

    G = nx.Graph()
    for i in range(len(graph)):
        for j in range(i, len(graph)):
            if graph[i][j] == 1:
                G.add_edge(i, j)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(5, 2))
    nx.draw(G, pos, with_labels=True, font_weight='bold', node_color='yellow', edge_color='gray')
    if path:
        edges_in_path = [(path[i], path[(i + 1) % len(graph)]) for i in range(len(graph))]
        nx.draw_networkx_edges(G, pos, edgelist=edges_in_path, edge_color='red', width=2)
    plt.show()


In [ ]:
hamCycle(cycle_graph_instance.graph)

## Complexity Analysis

In [143]:
def generate_complete_graph(n):
    """
    Generate a complete graph with 'n' vertices using Graph class.
    """
    complete_graph = nx.complete_graph(n)
    complete_graph_instance = Graph(n)
    complete_graph_instance.add_edges_from_graph(complete_graph)
    complete_graph_instance.summary()
    return cycle_graph_instance.graph

def main(n_values):
    bt_runtimes = []
    bf_runtimes = []

    for n in n_values:
        # Generate a complete graph
        graph = generate_complete_graph(n)

        # Backtracking algorithm
        start_time = time.time()
        hamCycle(graph)
        bt_runtime = time.time() - start_time
        bt_runtimes.append(bt_runtime)

        # Brute force algorithm
        start_time = time.time()
        hamiltonian_cycle_bruteforce(graph)
        bf_runtime = time.time() - start_time
        bf_runtimes.append(bf_runtime)

    # Plotting the runtimes
    plt.plot(n_values, bt_runtimes, label='Backtracking')
    plt.plot(n_values, bf_runtimes, label='Brute Force')
    plt.xlabel('Number of Vertices')
    plt.ylabel('Runtime (seconds)')
    plt.title('Comparison of Backtracking and Brute Force for Hamiltonian Cycle')
    plt.legend()
    plt.show()

# Example values for 'n'
n_values = [100]
main(n_values)

Nodes:  100
Edges:  4950


AttributeError: 'list' object has no attribute 'summary'

## Dynamic Programming